In [1]:
import pandas as pd


In [2]:
# Load the cleaned dataset produced in Phase 2
df = pd.read_csv("../01_raw_data/Luxury_Housing_Bangalore_CLEAN.csv")


In [3]:
# Quick sanity check to confirm data loaded correctly
df.head()


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,sarjapur road,Project_0,rmz,4025.0,4bhk,12.750846,primary,nri,2025-03-31,7.990091,5.462863,launch,broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,indiranagar,Project_1,puravankara,5760.0,3bhk,16.292152,primary,other,2024-06-30,4.839024,7.499123,under construction,nri desk,no,7.723898,106,not_provided
2,PROP000003,bannerghatta road,Project_2,tata housing,7707.0,4bhk,10.517724,primary,hni,2023-12-31,8.131315,8.669227,ready to move,direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,embassy,6192.0,3bhk,9.396367,primary,hni,2024-03-31,7.501657,5.720246,ready to move,online,yes,6.100929,106,Excellent location!
4,PROP000005,koramangala,Project_4,snn raj,7147.0,4bhk,15.345392,secondary,hni,2024-12-31,4.525216,8.609649,under construction,broker,no,5.312510,18,Too far from my office.


In [4]:
# Calculate price per square foot using ticket price and unit size
df["Price_per_Sqft"] = (df["Ticket_Price_Cr"] * 1e7) / df["Unit_Size_Sqft"]


In [5]:
# Verify the new column
df[["Ticket_Price_Cr", "Unit_Size_Sqft", "Price_per_Sqft"]].head()


,Ticket_Price_Cr,Unit_Size_Sqft,Price_per_Sqft
0,12.750846,4025.0,31679.120594
1,16.292152,5760.0,28284.985887
2,10.517724,7707.0,13646.976013
3,9.396367,6192.0,15175.012103
4,15.345392,7147.0,21471.096187


In [6]:
# Create booking flag based on transaction type
df["Booking_Flag"] = df["Transaction_Type"].map(
    {"Primary": 1, "Secondary": 0}
)


In [7]:
# Validate booking flag distribution
df["Booking_Flag"].value_counts()


Series([], Name: count, dtype: int64)

In [8]:
# Convert purchase quarter column to datetime
df["Purchase_Quarter"] = pd.to_datetime(df["Purchase_Quarter"])


In [9]:
# Extract quarter number from purchase date
df["Quarter_Number"] = df["Purchase_Quarter"].dt.quarter


In [10]:
# Verify quarter extraction
df[["Purchase_Quarter", "Quarter_Number"]].head()


,Purchase_Quarter,Quarter_Number
0,2025-03-31,1
1,2024-06-30,2
2,2023-12-31,4
3,2024-03-31,1
4,2024-12-31,4


In [11]:
# Helper column for conversion rate calculations
df["Booking_Conversion_Helper"] = 1


In [12]:
# Quick check
df[["Booking_Flag", "Booking_Conversion_Helper"]].head()


,Booking_Flag,Booking_Conversion_Helper
0,NaN,1
1,NaN,1
2,NaN,1
3,NaN,1
4,NaN,1


In [13]:
# Inspect final structure, data types, and non-null counts
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Property_ID                100000 non-null  str           
 1   Micro_Market               100000 non-null  str           
 2   Project_Name               100000 non-null  str           
 3   Developer_Name             100000 non-null  str           
 4   Unit_Size_Sqft             100000 non-null  float64       
 5   Configuration              100000 non-null  str           
 6   Ticket_Price_Cr            90087 non-null   float64       
 7   Transaction_Type           100000 non-null  str           
 8   Buyer_Type                 100000 non-null  str           
 9   Purchase_Quarter           100000 non-null  datetime64[us]
 10  Connectivity_Score         100000 non-null  float64       
 11  Amenity_Score              100000 non-null  float64       
 12  

In [14]:
# Review statistical summary of engineered numeric columns
df[[
    "Ticket_Price_Cr",
    "Unit_Size_Sqft",
    "Price_per_Sqft",
    "Booking_Flag",
    "Quarter_Number"
]].describe()


,Ticket_Price_Cr,Unit_Size_Sqft,Price_per_Sqft,Booking_Flag,Quarter_Number
count,90087.000000,100000.000000,90087.000000,0.0,100000.000000
mean,12.719069,6005.342680,23041.413502,NaN,2.497820
std,7.670383,1638.258325,16097.987842,NaN,1.117286
min,-1.420000,3000.000000,-3533.217218,NaN,1.000000
25%,10.010969,4683.000000,15523.086078,NaN,1.000000
50%,12.038456,6008.000000,20037.246546,NaN,2.000000
75%,14.097009,7332.000000,26616.036871,NaN,3.000000
max,100.000000,8999.000000,324359.390204,NaN,4.000000


In [15]:
# Save the final dataset for SQL loading and Power BI
final_path = "../01_raw_data/Luxury_Housing_Bangalore_FINAL.csv"
df.to_csv(final_path, index=False)


In [16]:
# Confirmation message
print("Final feature-engineered dataset saved successfully.")


Final feature-engineered dataset saved successfully.


In [17]:
# Replace empty strings with NaN for MySQL compatibility
import numpy as np

df.replace("", np.nan, inplace=True)

# Ensure numeric columns are numeric
numeric_cols = [
    "Ticket_Price_Cr",
    "Unit_Size_Sqft",
    "Price_per_Sqft",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [18]:
df.to_csv("../01_raw_data/Luxury_Housing_Bangalore_FINAL.csv", index=False)


In [19]:


# Convert empty strings to NaN across entire dataframe
df = df.replace(r'^\s*$', None, regex=True)

# Ensure numeric columns are proper numeric (invalid → NaN)
numeric_cols = [
    "Ticket_Price_Cr",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Price_per_Sqft",
    "Booking_Flag",
    "Quarter_Number"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# FINAL CHECK
df[numeric_cols].isnull().sum()


Ticket_Price_Cr           9913
Unit_Size_Sqft               0
Connectivity_Score           0
Amenity_Score                0
Locality_Infra_Score         0
Avg_Traffic_Time_Min         0
Price_per_Sqft            9913
Booking_Flag            100000
Quarter_Number               0
dtype: int64

In [20]:
# --- FIX BOOKING FLAG (MANDATORY FOR SQL) ---

# Since booking data is missing, assume not booked
df["Booking_Flag"] = 0

# Ensure correct type
df["Booking_Flag"] = df["Booking_Flag"].astype(int)

# Final sanity check
df["Booking_Flag"].value_counts()


Booking_Flag
0    100000
Name: count, dtype: int64

In [21]:
# Save SQL-safe CSV for MySQL import
sql_path = "../01_raw_data/Luxury_Housing_Bangalore_SQL.csv"
df.to_csv(sql_path, index=False)


In [23]:
numeric_cols = [
    "Ticket_Price_Cr",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Price_per_Sqft",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]

# Force numeric conversion (kills hidden strings)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# CRITICAL: replace NaN with actual None (SQL NULL)
df[numeric_cols] = df[numeric_cols].astype(object)
df[numeric_cols] = df[numeric_cols].where(df[numeric_cols].notnull(), None)

# Final validation
df[numeric_cols].isnull().sum()


Ticket_Price_Cr              9913
Unit_Size_Sqft                  0
Connectivity_Score              0
Amenity_Score                   0
Locality_Infra_Score            0
Avg_Traffic_Time_Min            0
Price_per_Sqft               9913
Booking_Flag                    0
Quarter_Number                  0
Booking_Conversion_Helper       0
dtype: int64

In [24]:
sql_path = "../01_raw_data/Luxury_Housing_Bangalore_SQL.csv"
df.to_csv(sql_path, index=False)


In [25]:
# List of numeric columns that must be SQL-safe
numeric_cols = [
    "Ticket_Price_Cr",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Price_per_Sqft",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]

# Force numeric conversion and replace invalid values with NaN
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# IMPORTANT: Replace NaN with actual NULL-compatible empty values
df[numeric_cols] = df[numeric_cols].where(df[numeric_cols].notnull(), None)

# Save a SQL-safe CSV (THIS is the file to import)
sql_safe_path = "../01_raw_data/Luxury_Housing_Bangalore_SQL_SAFE.csv"
df.to_csv(sql_safe_path, index=False)

print("SQL-safe CSV created successfully.")


SQL-safe CSV created successfully.


In [26]:
# List of numeric columns that must be SQL-safe
numeric_cols = [
    "Ticket_Price_Cr",
    "Price_per_Sqft",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]

# Force numeric conversion; invalid values become NaN
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# IMPORTANT: keep NaN (do NOT fill with "")
df[numeric_cols].isnull().sum()


Ticket_Price_Cr              9913
Price_per_Sqft               9913
Unit_Size_Sqft                  0
Connectivity_Score              0
Amenity_Score                   0
Locality_Infra_Score            0
Avg_Traffic_Time_Min            0
Booking_Flag                    0
Quarter_Number                  0
Booking_Conversion_Helper       0
dtype: int64

In [28]:
df.to_csv(
    "Luxury_Housing_Bangalore_SQL_SAFE.csv",
    index=False,
    na_rep=""  # EMPTY CELL = SQL NULL (critical)
)


In [31]:
import pandas as pd
import numpy as np

# Load original CLEAN file (not SQL_SAFE)
df = pd.read_csv("/Users/sai/Desktop/luxury_housing/01_raw_data/Luxury_Housing_Bangalore_CLEAN.csv")


# Numeric columns that caused MySQL failure
numeric_cols = [
    "Ticket_Price_Cr",
    "Price_per_Sqft",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]

# Force numeric conversion — invalid values → NaN
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# CRITICAL: ensure NaN stays NaN (do NOT convert to empty string)
df[numeric_cols].isnull().sum()


KeyError: 'Price_per_Sqft'

In [32]:
df.columns.tolist()


['Property_ID',
 'Micro_Market',
 'Project_Name',
 'Developer_Name',
 'Unit_Size_Sqft',
 'Configuration',
 'Ticket_Price_Cr',
 'Transaction_Type',
 'Buyer_Type',
 'Purchase_Quarter',
 'Connectivity_Score',
 'Amenity_Score',
 'Possession_Status',
 'Sales_Channel',
 'NRI_Buyer',
 'Locality_Infra_Score',
 'Avg_Traffic_Time_Min',
 'Buyer_Comments']

In [33]:
# Standardize column names: strip spaces + enforce consistent naming
df.columns = (
    df.columns
      .str.strip()
      .str.replace(" ", "_")
)


In [34]:
df.columns.tolist()


['Property_ID',
 'Micro_Market',
 'Project_Name',
 'Developer_Name',
 'Unit_Size_Sqft',
 'Configuration',
 'Ticket_Price_Cr',
 'Transaction_Type',
 'Buyer_Type',
 'Purchase_Quarter',
 'Connectivity_Score',
 'Amenity_Score',
 'Possession_Status',
 'Sales_Channel',
 'NRI_Buyer',
 'Locality_Infra_Score',
 'Avg_Traffic_Time_Min',
 'Buyer_Comments']

In [35]:
numeric_cols = [
    "Ticket_Price_Cr",
    "Price_per_Sqft",
    "Unit_Size_Sqft",
    "Connectivity_Score",
    "Amenity_Score",
    "Locality_Infra_Score",
    "Avg_Traffic_Time_Min",
    "Booking_Flag",
    "Quarter_Number",
    "Booking_Conversion_Helper"
]


In [36]:
set(numeric_cols) - set(df.columns)


{'Booking_Conversion_Helper',
 'Booking_Flag',
 'Price_per_Sqft',
 'Quarter_Number'}

In [39]:
# Ensure all SQL-required columns exist
required_columns = [
    'Property_ID',
    'Micro_Market',
    'Project_Name',
    'Developer_Name',
    'Unit_Size_Sqft',
    'Configuration',
    'Ticket_Price_Cr',
    'Transaction_Type',
    'Buyer_Type',
    'Purchase_Quarter',
    'Connectivity_Score',
    'Amenity_Score',
    'Possession_Status',
    'Sales_Channel',
    'NRI_Buyer',
    'Locality_Infra_Score',
    'Avg_Traffic_Time_Min',
    'Buyer_Comments',
    'Price_per_Sqft',
    'Booking_Flag',
    'Quarter_Number',
    'Booking_Conversion_Helper'
]

missing = set(required_columns) - set(df.columns)
missing


{'Booking_Conversion_Helper',
 'Booking_Flag',
 'Price_per_Sqft',
 'Quarter_Number'}

In [40]:
# Create missing columns explicitly

df['Booking_Flag'] = None
df['Quarter_Number'] = None
df['Booking_Conversion_Helper'] = 1  # default helper value

# Price per sqft must be computed, not left missing
df['Price_per_Sqft'] = df['Ticket_Price_Cr'] * 1e7 / df['Unit_Size_Sqft']


In [41]:
set(required_columns) - set(df.columns)


set()

In [42]:
numeric_cols = [
    'Ticket_Price_Cr',
    'Unit_Size_Sqft',
    'Connectivity_Score',
    'Amenity_Score',
    'Locality_Infra_Score',
    'Avg_Traffic_Time_Min',
    'Price_per_Sqft',
    'Booking_Flag',
    'Quarter_Number',
    'Booking_Conversion_Helper'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [43]:
df = df[required_columns]


In [44]:
df.to_csv(
    "Luxury_Housing_Bangalore_SQL_SAFE.csv",
    index=False,
    na_rep="NULL"
)
